In [44]:
import pandas as pd
import time 
import requests
import json 
from concurrent.futures import as_completed, ProcessPoolExecutor
from requests_futures.sessions import FuturesSession
from requests.adapters import HTTPAdapter
import time
from urllib3.util.retry import Retry
from datetime import datetime
import traceback 

with open('../Combine Data/api_key.txt', 'r') as file:
    api_key = file.read().strip()

In [33]:
Kiin_puuid = "Vpq3Y6Nns_bME-adpgmXfI89CIH3k0MCfDbrWZN2ASTuE8FlBit7rwbz3kzy_t4T8kgpGAo51asizA"
Canyon_puuid = "C7SQvoTcKkF1B2dZxm9Oo5yiRFtp7M5t0QNur_5jiPm1EymkWiVlTQAISnUeqmvqOQcm1QvKjTIYkg"
Chovy_puuid = "0ZeAFCtsYyae-HzXEvKujLs8XQUG-1fUjJiybfBAW1cPHR9ZPMU0hiQlafxPcyNDYkiYW4J-pBkMHQ"
Peyz_puuid  = "KSM9lFqeaD6TAzg5c2hkxxbMhJ01-6d_w6dNNiCaY-vsPRcjRbgXjq1EjhiaPpvW4Da07vFeduKkRQ"
Lehends_puuid = "sbe811AZXiBHw3UDQFn8TMHINYaKCNrMJQM0xUb-K7wBgHVvZ2p0EXVAOtWE-agBX3CNxyMwtjwuwA"

list_puuid = [Kiin_puuid,Canyon_puuid,Chovy_puuid,Peyz_puuid,Lehends_puuid]
list_nickname = ['Kiin','Canyon','Chovy','Peyz','Lehends']

In [31]:
dict_soloQ_ids = {}

In [32]:
def api_get_match_history_ids(puuid=None, region='asia', queue=420, start=0, count=100):
    """Gets the match history ids for a given puuid.

    Args:
        puuid (str, optional): Player's puuid. Defaults to None.
        region (str, optional): Player's region. Defaults to 'americas'.
        queue (int, optional): Queue ID. Defaults to 420 (Ranked).
        start (int, optional): Match # start (for pagination). Defaults to 0.
        count (int, optional): How many matches per page. Defaults to 100.

    Returns:
        list: List of match ids.
    """

    try:
        root_url = f'https://{region}.api.riotgames.com'
        endpoint = f'/lol/match/v5/matches/by-puuid/{puuid}/ids'
        query_params = f'?queue={queue}&start={start}&count={count}'

        

        while True:
            response = requests.get(root_url+endpoint+query_params+'&'+api_key)
            print(response.status_code)
            if response.status_code == 429:  # Rate limit exceeded
                retry_after = int(response.headers.get('Retry-After', 10))  # Default to 10 seconds
                print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
                time.sleep(retry_after)
            elif response.status_code == 200:  # Request successful
                return response.json()
            else:
                print(f"Unexpected status code: {response.status_code}. Retrying...")
                time.sleep(5)  # Wait for 5 seconds before retrying
    except Exception as e:
        print(f"An error occurred: {e}")
        return False


In [34]:
for puuid,nickname in zip(list_puuid,list_nickname):
    list_100 = api_get_match_history_ids(puuid) 
    list_200 = api_get_match_history_ids(puuid, start = 101) 
    list_300 = api_get_match_history_ids(puuid, start = 201) 
    list_400 = api_get_match_history_ids(puuid, start = 301) 
    list_500 = api_get_match_history_ids(puuid, start = 401) 
    dict_soloQ_ids[nickname] = list_100 + list_200 + list_300 + list_400 + list_500
    


200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


In [54]:
list_soloQ_ids_all_players = [value for sublist in dict_soloQ_ids.values() for value in sublist]
len(list_soloQ_ids_all_players)

2500

In [56]:
def process_match_json_reporting(match_json):
    """Processes the match json into a dataframe.

    Args:
        match_json (dict): Match JSON.
        puuid (str): Player's puuid.

    Returns:
        dataframe: Dataframe of the processed match data.
    """
    side_dict = {
        100:'blue',
        200:'red'
    }
    info = match_json['info']
    metadata = match_json['metadata']
    matchId = metadata['matchId']
    participants = metadata['participants']
    
    matchDFs = pd.DataFrame()
    
    for player_puuid in participants:
             
        player = info['participants'][participants.index(player_puuid)]

        #get opponent champion 
        for other_player in info['participants']:
            if other_player['teamPosition'] == player['teamPosition'] and other_player['championName'] != player['championName']:
                opp_champion = other_player['championName']

        gameCreation = info['gameCreation']
        gameStartTimestamp = info['gameStartTimestamp']
        gameEndTimestamp = info['gameEndTimestamp']
        timePlayed = gameEndTimestamp-gameStartTimestamp
        gameMode = info['gameMode']
        gameVersion = info['gameVersion']
        platformId = info['platformId']
        queueId = info['queueId']
        puuid = player['puuid']
        riotIdGameName = player['summonerName']
        try:
            riotIdTagLine = player['riotIdTagline']
        except:
            riotIdTagLine = ''
        side = side_dict[player['teamId']]
        win = player['win']

        champion = player['championName']
        kills = player['kills']
        deaths = player['deaths']
        assists = player['assists']
        summOne = player['summoner1Id']
        summTwo = player['summoner2Id']
        earlySurrender = player['gameEndedInEarlySurrender']
        surrender = player['gameEndedInSurrender']
        firstBlood = player['firstBloodKill']
        firstBloodAssist = player['firstBloodAssist']
        firstTower = player['firstTowerKill']
        firstTowerAssist = player['firstTowerAssist']
        dragonKills = player['dragonKills']

        damageDealtToBuildings = player['damageDealtToBuildings']
        damageDealtToObjectives = player['damageDealtToObjectives']
        damageSelfMitigated = player['damageSelfMitigated']
        goldEarned = player['goldEarned']
        teamPosition = player['teamPosition']
        lane = player['lane']
        largestKillingSpree = player['largestKillingSpree']
        longestTimeSpentLiving = player['longestTimeSpentLiving']
        objectivesStolen = player['objectivesStolen']
        totalMinionsKilled = player['totalMinionsKilled']
        totalAllyJungleMinionsKilled = player['totalAllyJungleMinionsKilled']
        totalEnemyJungleMinionsKilled = player['totalEnemyJungleMinionsKilled']
        totalNeutralMinionsKilled = totalAllyJungleMinionsKilled + totalEnemyJungleMinionsKilled
        totalDamageDealtToChampions = player['totalDamageDealtToChampions']
        totalDamageShieldedOnTeammates = player['totalDamageShieldedOnTeammates']
        totalHealsOnTeammates = player['totalHealsOnTeammates']
        totalDamageTaken = player['totalDamageTaken']
        totalTimeCCDealt = player['totalTimeCCDealt']
        totalTimeSpentDead = player['totalTimeSpentDead']
        turretKills = player['turretKills']
        turretsLost = player['turretsLost']
        visionScore = player['visionScore']
        controlWardsPlaced = player['detectorWardsPlaced']
        wardsKilled = player['wardsKilled']
        wardsPlaced = player['wardsPlaced']

        item0 = player['item0']
        item1 = player['item1']
        item2 = player['item2']
        item3 = player['item3']
        item4 = player['item4']
        item5 = player['item5']
        item6 = player['item6']
        try:
            perks = player['perks']

            perkKeystone = perks['styles'][0]['selections'][0]['perk']
            perkPrimaryRow1 = perks['styles'][0]['selections'][1]['perk']
            perkPrimaryRow2 = perks['styles'][0]['selections'][2]['perk']
            perkPrimaryRow3 = perks['styles'][0]['selections'][3]['perk']
            perkPrimaryStyle = perks['styles'][0]['style']
            perkSecondaryRow1 = perks['styles'][1]['selections'][0]['perk']
            perkSecondaryRow2 = perks['styles'][1]['selections'][1]['perk']
            perkSecondaryStyle = perks['styles'][1]['style']
            perkShardDefense = perks['statPerks']['defense']
            perkShardFlex = perks['statPerks']['flex']
            perkShardOffense = perks['statPerks']['offense']
        except:
            perkKeystone = ''
            perkPrimaryRow1 = ''
            perkPrimaryRow2 = ''
            perkPrimaryRow3 = ''
            perkPrimaryStyle = ''
            perkSecondaryRow1 = ''
            perkSecondaryRow2 = ''
            perkSecondaryStyle = ''
            perkShardDefense = ''
            perkShardFlex = ''
            perkShardOffense = ''


        matchDF = pd.DataFrame({
            'match_id': [matchId],
            'participants': [participants],
            'game_creation': [gameCreation],
            'game_start_timestamp': [gameStartTimestamp],
            'game_end_timestamp': [gameEndTimestamp],
            'game_version': [gameVersion],
            'queue_id': [queueId],
            'game_mode': [gameMode],
            'platform_id': [platformId],
            'puuid': [puuid],
            'riot_id': [riotIdGameName],
            'riot_tag': [riotIdTagLine],
            'time_played': [timePlayed],
            'side': [side],
            'win': [win],
            'team_position': [teamPosition],
            'lane': [lane],
            'champion': [champion],
            'kills': [kills],
            'deaths': [deaths],
            'assists': [assists],
            'summoner1_id': [summOne],
            'summoner2_id': [summTwo],
            'gold_earned': [goldEarned],
            'total_minions_killed': [totalMinionsKilled],
            'total_neutral_minions_killed': [totalNeutralMinionsKilled],
            'total_ally_jungle_minions_killed': [totalAllyJungleMinionsKilled],
            'total_enemy_jungle_minions_killed': [totalEnemyJungleMinionsKilled],
            'early_surrender': [earlySurrender],
            'surrender': [surrender],
            'first_blood': [firstBlood],
            'first_blood_assist': [firstBloodAssist],
            'first_tower': [firstTower],
            'first_tower_assist': [firstTowerAssist],
            'damage_dealt_to_buildings': [damageDealtToBuildings],
            'turret_kills': [turretKills],
            'turrets_lost': [turretsLost],
            'damage_dealt_to_objectives': [damageDealtToObjectives],
            'dragonKills': [dragonKills],
            'objectives_stolen': [objectivesStolen],
            'longest_time_spent_living': [longestTimeSpentLiving],
            'largest_killing_spree': [largestKillingSpree],
            'total_damage_dealt_champions': [totalDamageDealtToChampions],
            'total_damage_taken': [totalDamageTaken],
            'total_damage_self_mitigated': [damageSelfMitigated],
            'total_damage_shielded_teammates': [totalDamageShieldedOnTeammates],
            'total_heals_teammates': [totalHealsOnTeammates],
            'total_time_crowd_controlled': [totalTimeCCDealt],
            'total_time_spent_dead': [totalTimeSpentDead],
            'vision_score': [visionScore],
            'wards_killed': [wardsKilled],
            'wards_placed': [wardsPlaced],
            'control_wards_placed': [controlWardsPlaced],
            'item0': [item0],
            'item1': [item1],
            'item2': [item2],
            'item3': [item3],
            'item4': [item4],
            'item5': [item5],
            'item6': [item6],
            'perk_keystone': [perkKeystone],
            'perk_primary_row_1': [perkPrimaryRow1],
            'perk_primary_row_2': [perkPrimaryRow2],
            'perk_primary_row_3': [perkPrimaryRow3],
            'perk_secondary_row_1': [perkSecondaryRow1],
            'perk_secondary_row_2': [perkSecondaryRow2],
            'perk_primary_style': [perkPrimaryStyle],
            'perk_secondary_style': [perkSecondaryStyle],
            'perk_shard_defense': [perkShardDefense],
            'perk_shard_flex': [perkShardFlex],
            'perk_shard_offense': [perkShardOffense],
            'opp_champion': [opp_champion],

        })
        matchDFs = pd.concat([matchDFs,matchDF]) 
    return matchDFs

def api_get_match_data_from_match_ids(list_matchIds, region='asia', debug=False, reporting_focus = False):
    """Gets the match history for a given riot_id and riot_tag.

    Args:
        list MatchIds(list): Matches's puuid.
        region (str, optional): Player's region. Defaults to 'americas'.
        debug (bool, optional): Whether or not to print out matchIds as they are processed. Defaults to False.
        reporting_focus (bool, optional): Whether or not to focus on only picks and winrate. Defaults to False.

    Returns:
        DataFrame: DataFrame of all matches.
    """

    # Set up a session with retry mechanisms
    session = FuturesSession(executor=ProcessPoolExecutor(max_workers=10))
    retries = 5
    status_forcelist = [429]
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        respect_retry_after_header=True,
        status_forcelist=status_forcelist,
    )

    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    df = pd.DataFrame()

    list_matchIds = list(set(list_matchIds))

    if len(list_matchIds) > 0:

        # If there are new matches to process, create asynchronous requests for match data
        futures = [session.get(f'https://{region}.api.riotgames.com/lol/match/v5/matches/{matchId}' + '?' + api_key) for matchId in list_matchIds]

        i = 0

        # Iterate through completed asynchronous requests
        for future in as_completed(futures):
            resp = future.result()
            try:
                print('Row :',resp.json()['metadata']['matchId'])
            except : 
                    print('Answer :', resp.json())
                
            if debug:
                # If debug is enabled, print match processing information
                t1 = time.time()
                if reporting_focus  :
                    df = pd.concat([df, process_match_json_teams(resp.json())])
                else:
                    df = pd.concat([df, process_match_json_reporting(resp.json())])

                t2 = time.time()
                
                print('a',resp.json()['metadata']['matchId'] + f' - {i} ({round(t2 - t1, 2)}s)')
         
                i += 1
            else:
                if reporting_focus  :
                    df = pd.concat([df, process_match_json_teams(resp.json())])
                else:
                    df = pd.concat([df, process_match_json_reporting(resp.json())])

        try:# Return the DataFrame containing information about the fetched matches
            df['game_creation'] = pd.to_datetime(df['game_creation'], unit='ms')
            df['game_start_timestamp'] = pd.to_datetime(df['game_start_timestamp'], unit='ms')
            df['game_end_timestamp'] = pd.to_datetime(df['game_end_timestamp'], unit='ms')
            df['time_played'] = df['time_played']/60000
        except: 
            None
        return df
    else:
        # If there are no new matches to process, print a message and return an empty DataFrame
        print(f'No matches')
        return pd.DataFrame()
    

In [58]:
df_soloq_geng = api_get_match_data_from_match_ids(list_soloQ_ids_all_players)

Row : KR_7007718212
Row : KR_7004465435
Row : KR_6863080004
Row : KR_6969156239
Row : KR_6924999699
Row : KR_6996303807
Row : KR_6895664051
Row : KR_7002625094
Row : KR_6959054091
Row : KR_6992919667
Row : KR_6973639395
Row : KR_6942674385
Row : KR_6917583201
Row : KR_6965127631
Row : KR_6990499964
Row : KR_6928588930
Row : KR_6925035512
Row : KR_7003809659
Row : KR_6984865264
Row : KR_7014832369
Row : KR_6898828578
Row : KR_7022725550
Row : KR_6967547223
Row : KR_6949435163
Row : KR_6908408585
Row : KR_6944463977
Row : KR_6832402586
Row : KR_6845912159
Row : KR_6895493924
Row : KR_7025665076
Row : KR_7001431107
Row : KR_6973606943
Row : KR_6958924073
Row : KR_6924934681
Row : KR_6835340294
Row : KR_6831029570
Row : KR_7024348802
Row : KR_7025176492
Row : KR_6993933935
Row : KR_6980615629
Row : KR_6925057411
Row : KR_7009936037
Row : KR_6917623365
Row : KR_6908789697
Row : KR_6746652090
Row : KR_6966443933
Row : KR_6990579139
Row : KR_6987649961
Row : KR_6731298844
Row : KR_6913426099


ConnectionError: None: Max retries exceeded with url: /lol/match/v5/matches/KR_6911400399?api_key=RGAPI-749a8ccc-def6-44d1-beb4-01d7af2546e5 (Caused by None)

In [47]:
def compute_game_gold_per_dmg_avg(dict_timeline):


    """
    Calculate the gold per dmg per minute avg for the game sent for each players's puuid

    Args:
        dict_timeline (dict): dict_timeline, timeline data in json format of a match

    Returns:
        dataFrame: dataframe of all players's puuid gold/dmg per minute ratio 
    """ 
    dict_per_frame_gold_dmg = {}
    list_dict_xp_diff = []

    for frames in range (len(dict_timeline['info']['frames'])): # go through frames of the game 

        ###### DMG / GOLD PER MINUTE ######
        dict_frame_gold_dmg = {}
        
        #
        for participants_id in dict_timeline['info']['frames'][frames]['participantFrames'].keys():
            list_ratio = []
            participants_puuid = dict_timeline['info']['participants'][int(participants_id)-1]['puuid']
            if frames > 4:
                dict_frame_gold_dmg[participants_puuid] = (dict_timeline['info']['frames'][frames]['participantFrames'][participants_id]['damageStats']['totalDamageDoneToChampions']/dict_timeline['info']['frames'][frames]['participantFrames'][participants_id]['totalGold']) if dict_timeline['info']['frames'][frames]['participantFrames'][participants_id]['damageStats']['totalDamageDoneToChampions']> 0 else 0
        dict_per_frame_gold_dmg[frames] = dict_frame_gold_dmg

        ###### XP DIFF OPPONENT 5 / 10 / 15  ######
        if frames == 5 or frames == 10 or frames == 15 :

            
            dict_xp_diff = {} # each frame xp diff for the game
            
            for participants_id in dict_timeline['info']['frames'][frames]['participantFrames'].keys():
                
                if int(participants_id)<6: #half of the participants
                    

                    participants_puuid = dict_timeline['info']['participants'][int(participants_id)-1]['puuid']
                    opponents_puuid = dict_timeline['info']['participants'][int(participants_id)-1+5]['puuid']
                    opponents_id = int(participants_id) + 5


                    
                    participants_xp = dict_timeline['info']['frames'][frames]['participantFrames'][participants_id]['xp']
                    opponent_xp = dict_timeline['info']['frames'][frames]['participantFrames'][str(opponents_id)]['xp'] 

                    part_opp_xpdiff = participants_xp - opponent_xp
                    opp_part_xpdiff = opponent_xp - participants_xp 

                    dict_xp_diff[participants_puuid] = part_opp_xpdiff
                    dict_xp_diff[opponents_puuid] = opp_part_xpdiff
                    

            
            list_dict_xp_diff.append(dict_xp_diff)


    ### GOLD 
    avg_data = {}
    # From dict_per_frame to dict puuid : avg gold
    for inner_dict in dict_per_frame_gold_dmg.values():
        
        for puuid, value in inner_dict.items():
            if puuid not in avg_data:
                avg_data[puuid] = value
            else:
                avg_data[puuid] += value

    # Calculate the average for each "puuid"
    for puuid, value in avg_data.items():
        avg_data[puuid] = value / (len(dict_per_frame_gold_dmg) - 5) # 5 first minutes doesnt count


    ### XP 
    if len(list_dict_xp_diff) == 3 : 
        #print('NO ERROR')
        df_xp = pd.DataFrame(list_dict_xp_diff)
    else : 
        print('ERROR')
        print(list_dict_xp_diff)
    # Add prefix to columns
    df_xp = df_xp.T
    df_xp.columns = ['xpdiff_at5', 'xpdiff_at10', 'xpdiff_at15']
    # Add 'puuid' column
    df_xp['puuid'] = list(list_dict_xp_diff[0].keys()) 
    df_xp['match_id'] = dict_timeline['metadata']['matchId']
    df_xp = df_xp[['match_id', 'puuid', 'xpdiff_at5', 'xpdiff_at10', 'xpdiff_at15']]
    ##########################################
 
    return pd.DataFrame(avg_data.items(), columns=['puuid', 'avg']), df_xp

def api_get_match_timeline_data(matchIds, region='asia', debug=False):
    """Gets the match timeline data for a given matchIds list 

    Args:
        
        matchIds (list of str): List of matchIds to study.
        region (str, optional): Player's region. Defaults to 'europe'.
        debug (bool, optional): Whether or not to print out matchIds as they are processed. Defaults to False.

    Returns:
        list of Json : list of json of timeline matches data
    """

    # Set up a session with retry mechanisms
    session = FuturesSession(executor=ProcessPoolExecutor(max_workers=10))
    retries = 5
    status_forcelist = [429]
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        respect_retry_after_header=True,
        status_forcelist=status_forcelist,
    )

    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)


    futures = []
    print('Nombre de matchs à compute:',len(matchIds))
    # If there are new matches to process, create asynchronous requests for match data
    futures = [session.get(f'https://{region}.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline' + '?' + api_key) for matchId in matchIds]
                    
    i = 0
    list_data = []
    # Iterate through completed asynchronous requests
    for future in as_completed(futures):
        resp = future.result()
        try :
            print('Row :',resp.json()['metadata']['matchId'])
            list_data.append(resp.json())
        except :
            print(resp.json())
            print('Error while retrieving ')

    return list_data

In [48]:
list_data_game_timeline = api_get_match_timeline_data(dict_soloQ_ids['Kiin'])
df_gold = pd.DataFrame()
df_xpdiff = pd.DataFrame()

for game_data in list_data_game_timeline : 

    df_gold_temp, df_xpdiff_temp  = compute_game_gold_per_dmg_avg(game_data)
    df_gold = pd.concat([df_gold,df_gold_temp])
    df_xpdiff = pd.concat([df_xpdiff,df_xpdiff_temp])
    

df_gold_dmg = df_gold.groupby('puuid').mean().reset_index()
df_xpdiff

Nombre de matchs à compute: 500
Row : KR_7025608775
Row : KR_7025492701
Row : KR_7025373497
Row : KR_7025312048
Row : KR_7025221690
Row : KR_7025575238
Row : KR_7025085038
Row : KR_7025053009
Row : KR_7025187216
Row : KR_7025142201
Row : KR_7024500899
Row : KR_7024472560
Row : KR_7024438284
Row : KR_7024398325
Row : KR_7024343217
Row : KR_7024262575
Row : KR_7024194262
Row : KR_7024108504
Row : KR_7023321180
Row : KR_7022923921
Row : KR_7022908003
Row : KR_7022857121
Row : KR_7022770461
Row : KR_7022725550
Row : KR_7022800820
Row : KR_7022644187
Row : KR_7021850985
Row : KR_7019796941
Row : KR_7021841041
Row : KR_7019858770
Row : KR_7015701893
Row : KR_7019645861
Row : KR_7019687509
Row : KR_7015674568
Row : KR_7015731723
Row : KR_7015622517
Row : KR_7014848826
Row : KR_7014832369
Row : KR_7012559223
Row : KR_7012208354
Row : KR_7012524135
Row : KR_7012235347
Row : KR_7012020955
Row : KR_7012098465
Row : KR_7012143529
Row : KR_7011934420
Row : KR_7011119294
Row : KR_7011072230
Row : KR

UnboundLocalError: cannot access local variable 'df_xp' where it is not associated with a value

In [ ]:
# match error : KR_6761812978